# Demo the model inspection capability

- Get the input and output.
- Steer from the input and output.

In [1]:
from dawnet.inspector import LLMInspector
from dawnet import op

In [2]:
text = "When John and Mary went to the store, John gave the bag to"

### Load the model from Huggingface

In [3]:
insp = LLMInspector.from_hf("Qwen/Qwen3-4B")
print(insp.model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2560,), eps=1e-06)
        (post_attention_layernorm): Qwe

### Get the input and output of all layers

In [4]:
_ = insp.add(op.GetOutput(), name_regex=r"model.layers.\d+$")

Added to layer ['model.layers.0', 'model.layers.1', 'model.layers.2', 'model.layers.3', 'model.layers.4', 'model.layers.5', 'model.layers.6', 'model.layers.7', 'model.layers.8', 'model.layers.9', 'model.layers.10', 'model.layers.11', 'model.layers.12', 'model.layers.13', 'model.layers.14', 'model.layers.15', 'model.layers.16', 'model.layers.17', 'model.layers.18', 'model.layers.19', 'model.layers.20', 'model.layers.21', 'model.layers.22', 'model.layers.23', 'model.layers.24', 'model.layers.25', 'model.layers.26', 'model.layers.27', 'model.layers.28', 'model.layers.29', 'model.layers.30', 'model.layers.31', 'model.layers.32', 'model.layers.33', 'model.layers.34', 'model.layers.35']


In [5]:
with insp.ctx(detach_state=True) as state:
  out = insp.infer(text, chat=False)

In [6]:
out.topk(5)

(tensor([18.8178, 18.4397, 16.6338, 16.2505, 13.9637], device='mps:0',
        grad_fn=<TopkBackward0>),
 tensor([10244,   279,   264,   806, 29405], device='mps:0'),
 [' Mary', ' the', ' a', ' his', ' Alice'])

#### The layer output is stored in `state` variable, and can be accessed as follow

In [7]:
for key, value in state['output'].items():
  print(key, value.shape)

model.layers.0 torch.Size([1, 14, 2560])
model.layers.1 torch.Size([1, 14, 2560])
model.layers.2 torch.Size([1, 14, 2560])
model.layers.3 torch.Size([1, 14, 2560])
model.layers.4 torch.Size([1, 14, 2560])
model.layers.5 torch.Size([1, 14, 2560])
model.layers.6 torch.Size([1, 14, 2560])
model.layers.7 torch.Size([1, 14, 2560])
model.layers.8 torch.Size([1, 14, 2560])
model.layers.9 torch.Size([1, 14, 2560])
model.layers.10 torch.Size([1, 14, 2560])
model.layers.11 torch.Size([1, 14, 2560])
model.layers.12 torch.Size([1, 14, 2560])
model.layers.13 torch.Size([1, 14, 2560])
model.layers.14 torch.Size([1, 14, 2560])
model.layers.15 torch.Size([1, 14, 2560])
model.layers.16 torch.Size([1, 14, 2560])
model.layers.17 torch.Size([1, 14, 2560])
model.layers.18 torch.Size([1, 14, 2560])
model.layers.19 torch.Size([1, 14, 2560])
model.layers.20 torch.Size([1, 14, 2560])
model.layers.21 torch.Size([1, 14, 2560])
model.layers.22 torch.Size([1, 14, 2560])
model.layers.23 torch.Size([1, 14, 2560])
mo

### Modify the output of a layer

For example, we copy the layer 6 of token Mary to layer 6 of the second token John. We would hope that the model will complete with "John" rather than "Mary".

In [8]:
tokenized = insp.tokenizer.tokenize(text)
for idx, tok in enumerate(tokenized):
  print(idx, tok)

0 When
1 ĠJohn
2 Ġand
3 ĠMary
4 Ġwent
5 Ġto
6 Ġthe
7 Ġstore
8 ,
9 ĠJohn
10 Ġgave
11 Ġthe
12 Ġbag
13 Ġto


In [9]:
# Remove previous GetOutput operation
insp.remove_all()

# Attach a hook
set_op = insp.add(op.SetOutput(), name="model.layers.6")
def set_callback(obj):
  obj[0,9] = obj[0,3]
  return obj

with insp.ctx([set_op.run_params(output_fn=set_callback)]):   # Attach the callback function
  out2 = insp.infer(text, chat=False)

Added to layer ['model.layers.6']


In [10]:
out2.topk(5)

(tensor([19.0902, 18.4072, 16.2737, 15.2673, 14.9571], device='mps:0',
        grad_fn=<TopkBackward0>),
 tensor([ 3757,   279,   264,  7801, 23223], device='mps:0'),
 [' John', ' the', ' a', ' James', ' Anna'])